In [1]:
#Import Modules

#numpy
import numpy as np
from numpy.random import multivariate_normal #For later example

import pandas as pd

#advectionGP
from advectionGP.models import AdjointAdvectionDiffusionReactionModel as PDEModel #Model module builds basic parts of the PDE problem, combines other classes into full model
from advectionGP.models import AdjointSimpleODEModel as ODEModel #Model module builds basic pa
from advectionGP.sensors import FixedSensorModel #Builds sensor arrays to generate data for foward model or to generate observations for comparison
from advectionGP.kernels import EQ #Generates exponentiated quadratic kernel approximation
from advectionGP.test import TestKernels #Unit test model
from advectionGP.wind import WindFixU #Wind model

#Plotting tools
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# GPy: Gaussian processes library
import GPy

import scipy

#GPyOpt
import GPyOpt
from GPyOpt.methods import BayesianOptimization

In [10]:
# generate sensor locations for training data with shape [total observations, 4], where each row has elements 
#[lower time location, upper time location, x location, y location]

tlocL = np.linspace(1,8,3) # lower time
xloc=np.linspace(2,8,5) # x locations
yloc=np.linspace(2,8,5) # y locations
sensN = len(xloc)*len(yloc) # total number of sensors 
obsN = len(tlocL) # total time points at which an observation is taken
X= np.zeros((obsN*sensN,4)) # obsN*sensN is total observations over all sensors and all times
# Build sensor locations
X[:,0] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[0] #lower time
X[:,2] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[1] # x location
X[:,3] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[2] # ylocation
X[:,1] = X[:,0]+1 # upper time

XGP= np.zeros((sensN*obsN,3))
XGP[:,0] = (X[:,0]+X[:,1])/2
XGP[:,1] = X[:,2]
XGP[:,2] = X[:,3] 



sensors = FixedSensorModel(X,1) # establish sensor model arguments are sensor locations and spatial averaging

# generate sensor locations for test data with shape [total observations, 4], where each row has elements 
#[lower time location, upper time location, x location, y location]

tlocL = np.linspace(2,9,5) # lower time
xloc=np.linspace(1.5,8.5,4) # x locations
yloc=np.linspace(1.5,8.5,4) # y locations
sensN = len(xloc)*len(yloc) # total number of sensors 
obsN = len(tlocL) # total time points at which an observation is taken
Xtest= np.zeros((obsN*sensN,4)) # obsN*sensN is total observations over all sensors and all times
# Build sensor locations
Xtest[:,0] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[0] 
Xtest[:,2] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[1]
Xtest[:,3] = np.asarray(np.meshgrid(tlocL,xloc,yloc)).reshape(3,sensN*obsN)[2]
Xtest[:,1] = Xtest[:,0]+1

XGPtest= np.zeros((sensN*obsN,3))
XGPtest[:,0] = (Xtest[:,0]+Xtest[:,1])/2
XGPtest[:,1] = Xtest[:,2]
XGPtest[:,2] = Xtest[:,3]

sensorsTest = FixedSensorModel(Xtest,1) # establish sensor model

In [11]:
k_0 = 0.01 #Diffusion
R=0.1
noiseSD = 0.05 #Observation noise
N_feat=2000 # number of features used to approximate ground truth GP
boundary = ([0,0,0],[10,10,10])# corners of the grid - in units of space
k = EQ(2, 2.0) # generate EQ kernel arguments are lengthscale and variance
res = [50,30,30] # grid size for time, x and y
wind=np.cos(np.linspace(0,6*np.pi,res[1]))*0.5
u=[]
u.append(np.ones(res)*0.01) #x direction wind
u.append(np.ones(res)*0.01) # y direction wind
windmodel=WindFixU(u)
#u.append(np.ones(res)*0.1) #x direction wind
#u.append(np.ones(res)*0.1) # y direction wind
m = PDEModel(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #initiate PDE model to build concentration

dt,dx,dy,dx2,dy2,Nt,Nx,Ny = m.getGridStepSize() # useful numbers!

z=np.random.normal(0,1.0,N_feat) # Generate z to compute source
sourceGT=(m.computeSourceFromPhi(z))# Compute source

In [12]:
concTrain=m.computeConcentration(sourceGT) # Compute concentration - runs advection diffusion forward model
yTrain= m.computeObservations() # Compute observations with noise uses m.sensormodel for observation locations

m.sensormodel=sensorsTest
yTest= m.computeObservations() 

In [13]:
k2 = EQ(2, 2.0) 

In [14]:
%%timeit -r 4 -n 10
mPDE = PDEModel(resolution=res,boundary=boundary,N_feat=10,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #Initiate new model for inference
regressPDE = mPDE.computeModelRegressors() # Compute regressor matrix
meanZPDE, covZPDE = mPDE.computeZDistribution(yTrain) # Infers z vector mean and covariance
sourcePDE = mPDE.computeSourceFromPhi(meanZPDE) # Generates estimated source using inferred distributio

Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 
Calculating Phis...
9/10 
Calculating Adjoints...
74/75 


In [16]:
import time

In [17]:
%%timeit -r 4 -n 10
N_feat = 10
k2 = EQ(2, 2.0)
sigmaMCMC=0.005
out2 = []
var = np.ones(len(yTrain))*noiseSD**2
accept = 0 
start_time2 = time.time()
t_end = time.time() +3.4
iters = 0
ObsN=75

mMCMC = PDEModel(resolution=res,boundary=boundary,N_feat=N_feat,noiseSD=noiseSD,kernel=k2,sensormodel=sensors,windmodel=windmodel,k_0=k_0,R=R) #initiate PDE model to build concentration
#zCur=np.random.normal(0,1.0,N_feat) # Generate z to compute source
zCur=np.ones(N_feat)
source = mMCMC.computeSourceFromPhi(zCur) # Generates estimated source using inferred distribution
conc=mMCMC.computeConcentration(source) # Compute test concentration
yMCMC= mMCMC.computeObservations() # Compute observations with noise
#llObsCur = -0.5*np.sum((np.square(np.subtract(yTrain,yMCMC))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
logPiCur = -np.inf
llObsCur = 0
iterations=20000
#out2.append(zCur)
#for zi in zCur:
#    logPriorCur += np.log(scipy.stats.norm(0, 1).pdf(zi))

for i in range(iterations):
    zCan = zCur + np.random.normal(0,sigmaMCMC,N_feat)
    source = mMCMC.computeSourceFromPhi(zCan) # Generates estimated source using inferred distribution
    concCan=mMCMC.computeConcentration(source) # Compute test concentration
    yCan= mMCMC.computeObservations() # Compute observations with noise
    llObsCan = -0.5*np.sum((np.square(np.subtract(yTrain,yCan))/(noiseSD**2)))-0.5*ObsN*np.log(noiseSD**2)-0.5*ObsN*np.log(2*np.pi)
    logPriorCan = 0
    for j in zCan:
        logPriorCan += np.log(scipy.stats.norm(0, 1).pdf(j))

    logPiCan = llObsCan + logPriorCan
    #logPiCur = llObsCur + logPriorCur
    #print(logPiCan,logPiCur)
    u = np.random.uniform(0,1)

    if np.log(u) < logPiCan - logPiCur:
        zCur = zCan
        #logPriorCur = logPriorCan
        logPiCur = logPiCan
        accept += 1
    out2.append(zCur)
    iters +=1
    if time.time() > t_end:
        break 
        
print(iters)

85
95
100
94
94
94
94
95
95
97
97
95
94
102
95
95
96
98
91
97
97
99
101
95
92
87
71
53
71
55
43
53
63
79
51
40
67
61
63
66
3.42 s ± 7.9 ms per loop (mean ± std. dev. of 4 runs, 10 loops each)
